In [1]:
import numpy as np
import glob
import os
import pandas as pd
import scipy.signal as signal
import mne
import torch
from torch.utils.data import DataLoader, Dataset
from mne import preprocessing, Epochs
import utils
import matplotlib.pyplot as plt

## Data pre-processing

### Band pass filtering and events

In [ ]:
eeg_file_path = 'data/eeg_data_A/'

eeg_training_files = glob.glob('data/eeg_data_A/A0*T.gdf')
eeg_eval_files = glob.glob(os.path.join(eeg_file_path, 'A0*E.gdf'))

print(eeg_training_files)
eeg_channels = 25

raw_eeg_objects = []  # list to store raw objects
eeg_epoch_objects = []

eeg_train_obj, epoch_train_obj = utils.band_pass_filter(eeg_training_files)
eeg_eval_obj, epoch_eval_obj = utils.band_pass_filter(eeg_eval_files)

Convert to PyTorch Tensors <br>
The size will be equal to [Number of EEG Recordings, Number of Channels, Number of Time Points]

In [3]:
eeg_train_data = utils.raw_to_tensor(eeg_train_obj)
eeg_eval_data = utils.raw_to_tensor(eeg_eval_obj)

for epoch in epoch_train_obj:
    print(epoch.event_id)

{'32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10, '276/32766': 11, '277/32766': 12, '1072/32766': 13, '1023/768': 14}
{'32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10, '276/32766': 11, '277/32766': 12, '1072/32766': 13, '1023/768': 14}
{'32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10, '276/32766': 11, '277/32766': 12, '1072/32766': 13, '1023/768': 14}
{'32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10, '276/32766': 11, '277/32766': 12, '1072/32766': 13, '1023/768': 14}
{'32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10, '276/32766': 11, '277/32766': 12, '1072/32766': 13, '1023/768': 14}
{'32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10, '276/32766': 11, '277/32766': 12, '1072/32766': 13, '1023/768': 14}
{'32766': 3, '768': 4, '769': 5, '770': 6, '771': 7, '772': 8, '1072/32766': 9, '1023/768': 10}
{'32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10, '276/32766': 11, '277/32766': 12, '1072/32766': 

Create training data by data augmentation

In [4]:
eeg_train_data = np.transpose(eeg_train_data)
eeg_train_data = np.expand_dims(eeg_train_data, axis=1)

In [5]:
class EEGDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __get_item__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label
